In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from dlomix.constants import ALPHABET_UNMOD
from dlomix.layers.attention import AttentionLayer
from prosit_t.layers import (
    MetaEncoder,
    FusionLayer,
    TransformerDecoder,
    Regressor,
    SequenceEncoder,
    SequenceEncoderNoGRU,
)
from prosit_t.models import PrositTransformerIntensityPredictorV2
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
from dlomix.data import IntensityDataset
from dlomix.constants import ALPHABET_UNMOD
import wandb
from wandb.keras import WandbCallback

2023-06-02 14:52:48.035509: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 14:52:49.123990: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
TRAIN_DATAPATH = "https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/Intensity/proteomeTools_train_val.csv"
EPOCHS=1

In [4]:
BATCH_SIZE = 64
int_data = IntensityDataset(
    data_source=TRAIN_DATAPATH,
    seq_length=30,
    collision_energy_col="collision_energy",
    batch_size=BATCH_SIZE,
    val_ratio=0.2,
    test=False,
)

2023-06-02 14:52:59.671365: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 28702 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:21:00.0, compute capability: 8.6
2023-06-02 14:52:59.671931: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 43610 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6
2023-06-02 14:52:59.672298: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 43610 MB memory:  -> device: 2, name: NVIDIA A40, pci bus id: 0000:e2:00.0, compute capability: 8.6


In [5]:
project_name = "transforming-prosit"
wandb.init(project=project_name, name='transformers_v2')

wandb: Currently logged in as: mamisashvili-lizi. Use `wandb login --relogin` to force relogin


In [6]:
model = PrositTransformerIntensityPredictorV2()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=masked_spectral_distance,
    metrics=[masked_pearson_correlation_distance],
)

5 -------------num encoders------------
6 len_fion


In [7]:
model.fit(
    int_data.train_data,
    validation_data=int_data.val_data,
    epochs=EPOCHS,
    callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
2023-06-02 14:53:02.345516: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [5900,6]
	 [[{{node Placeholder/_2}}]]
2023-06-02 14:53:02.345885: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [5900,1]
	 [[{{

(None, 1) (None, 6)
(None, 29, 16) regressor input size------
(None, 29, 6) regressor output size------
(None, 1) (None, 6)
(None, 29, 16) regressor input size------
(None, 29, 6) regressor output size------


2023-06-02 14:53:13.084572: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-02 14:53:13.114613: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fcbcf8a6580 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-02 14:53:13.114632: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2023-06-02 14:53:13.114637: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (1): NVIDIA A40, Compute Capability 8.6
2023-06-02 14:53:13.114641: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (2): NVIDIA A40, Compute Capability 8.6
2023-06-02 14:53:13.117990: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-02 14

367/369 [============================>.] - ETA: 0s - loss: 0.6727 - masked_pearson_correlation_distance: 0.5633(None, 1) (None, 6)
(None, 29, 16) regressor input size------
(None, 29, 6) regressor output size------


wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(None, 16), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fd3ec59bd60>, 140550658812896), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(16,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fd3ec5aec70>, 140550658694112), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(16, 4, 4), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fd3ec542b20>, 140550658397664), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(4, 4), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fd3ec5599d0>, 140550658397872), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(16, 4, 4), d

2023-06-02 14:53:43.148403: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,29,16]
	 [[{{node inputs}}]]
2023-06-02 14:53:43.152549: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,29,16]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230602_145301-7hyolqab/files/model-best/assets


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230602_145301-7hyolqab/files/model-best/assets
wandb: Adding directory to artifact (/cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230602_145301-7hyolqab/files/model-best)... Done. 0.1s


369/369 [==============================] - 44s 57ms/step - loss: 0.6726 - masked_pearson_correlation_distance: 0.5631 - val_loss: 0.6418 - val_masked_pearson_correlation_distance: 0.5133


In [8]:
model.summary()

------------------
(None, 1) (None, 6)
(None, 29, 16) regressor input size------
(None, 29, 6) regressor output size------
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 string_lookup (StringLookup)   (None, 30)           0           ['input_1[0][0]']                
                                                                                                  
 embedding (Embedding)          (None, 30, 16)       352         ['string_lookup[0][0]']          
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []   

In [10]:
for i in int_data.train_data:
    print(i[1])
    break

tf.Tensor(
[[ 0.          0.         -1.         ... -1.         -1.
  -1.        ]
 [ 0.01429778  0.          0.         ... -1.         -1.
  -1.        ]
 [ 0.          0.         -1.         ... -1.         -1.
  -1.        ]
 ...
 [ 0.03522496  0.         -1.         ... -1.         -1.
  -1.        ]
 [ 0.          0.         -1.         ... -1.         -1.
  -1.        ]
 [ 0.          0.          0.         ... -1.         -1.
  -1.        ]], shape=(64, 174), dtype=float32)
